In [1]:
import os

import pandas as pd
from grab_util import grab_from_url_json
import threading

In [2]:
apiKey = '7f59af901d2d86f78a1fd60c1bf9426a'

In [3]:
df = pd.read_csv('SUSTC_Journals/articles_all/all.csv')
len(df)

1230630

In [5]:
def grab_scopus_eid(doi_column, start_i, end_i):
    df_s = pd.DataFrame()

    for index, doi in enumerate(df[start_i:end_i][doi_column]):
        if index % 500  == 0:
            print(index)
        dfx = pd.DataFrame()
        eid = ''
        if doi != None:
            url = "https://api.elsevier.com/content/search/scopus?query=doi(%s)&apiKey=%s" % (doi, apiKey)
            res = grab_from_url_json(url, headers={'Accept': 'application/json'})
            if res is not None:
                if 'search-results' in res and 'entry' in res['search-results'] and len(res['search-results']['entry']) > 0:
                    entry = res['search-results']['entry'][0]
                    if 'eid' in entry:
                        eid = entry['eid']

        dfx['doi'] = [doi]
        dfx['eid'] = [eid]

        df_s = df_s.append(dfx, ignore_index=True)

        if index == 500:
            df_s.to_csv(f'SUSTC_Journals/articles_all/all_scopus_eid_{start_i + 1}-{end_i}_test.csv', index=False)
    df_s.to_csv(f'SUSTC_Journals/articles_all/all_scopus_eid_{start_i + 1}-{end_i}.csv', index=False)
    return

In [6]:
%%time
step = 10000
t_num = int(len(df) / step)

threads = []
for i in range(0, t_num):
    start_i = i*step
    end_i = start_i + step

    if end_i + step > len(df):
        end_i = len(df)
    
    threadx = threading.Thread(target=grab_scopus_eid, name=f'Thread{start_i}-{end_i}', args=('DI', start_i, end_i, ))
    threadx.start()
    threads.append(threadx)

for t in threads:
    t.join()

(read timeout=10)
ReadTimeout: HTTPSConnectionPool(host='api.elsevier.com', port=443): Read timed out. (read timeout=10)
ReadTimeout: HTTPSConnectionPool(host='api.elsevier.com', port=443): Read timed out. (read timeout=10)
ReadTimeout: HTTPSConnectionPool(host='api.elsevier.com', port=443): Read timed out. (read timeout=10)
ReadTimeout: HTTPSConnectionPool(host='api.elsevier.com', port=443): Read timed out. (read timeout=10)
ReadTimeout: HTTPSConnectionPool(host='api.elsevier.com', port=443): Read timed out. (read timeout=10)
ReadTimeout: HTTPSConnectionPool(host='api.elsevier.com', port=443): Read timed out. (read timeout=10)
ReadTimeout: HTTPSConnectionPool(host='api.elsevier.com', port=443): Read timed out. (read timeout=10)
ReadTimeout: HTTPSConnectionPool(host='api.elsevier.com', port=443): Read timed out. (read timeout=10)
ReadTimeout: HTTPSConnectionPool(host='api.elsevier.com', port=443): Read timed out. (read timeout=10)
ReadTimeout: HTTPSConnectionPool(host='api.elsevier.com

In [4]:
import os
file_list = os.listdir('SUSTC_Journals/articles_all/')

In [5]:

def grab_mendeley_views(eid_column, start_i, end_i, end_year=30000, end_month=12):
    if f'all_mendeley_{start_i + 1}-{end_i}.csv' in file_list:
        return

    df = pd.read_csv(f'SUSTC_Journals/articles_all/all_scopus_eid_{start_i + 1}-{end_i}.csv')
    df_s = pd.DataFrame()

    if 'views' not in df_s.columns:
        for index, eid in enumerate(df[eid_column]):
            if index % 500  == 0:
                print(index)
            if str(eid) == 'nan':
                continue
            dfx = pd.DataFrame()
            views = 0
            citations = 0
            if str(eid) != 'nan':
                url = 'https://www.mendeley.com/stats/articles/timeline/' + eid
                res = grab_from_url_json(url, headers={'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
                                                        'accept-encoding': 'gzip, deflate, br',
                                                        'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
                                                        'cache-control': 'max-age=0',
                                                        'cookie': '__cfduid=d00fe5ce2b8615a4fac8e00b06eda10891578447833; web_session=28322787-15b6-487f-8e9a-9730839d884b; msso=4%3A7bd2633e-be90-4e69-9bf5-35a9fbeb268c;aa_usr=%7B%22accessType%22%3A%22ae%3AREG%3AU_P%3AGUEST%3A%22%2C%22accountId%22%3A%22ae%3A293744%22%2C%22accountName%22%3A%22ae%3AMendeley%20Guests%22%2C%22userId%22%3A%22ae%3A34829421%22%7D; node_session=eyJpZCI6IjVhOTliMDRmLTkwMmUtNDRiMi05MGYwLWFiMGQxMjQ5YjU0MSIsInN0YXRlIjoiMmM5MWM4MjM0ZGJhNmM1NWIxZjk0OWViMDdkZmI2ZmYiLCJyZWZlcnJlclVybCI6Imh0dHA6Ly93d3cubWVuZGVsZXkuY29tL3N0YXRzL2FydGljbGVzL3RpbWVsaW5lLzItczIuMC04NDk2NDM5MDk2MyIsImlzU29mdFNpZ25JbiI6ZmFsc2UsInByb21wdCI6ImxvZ2luIiwiYW5hbHl0aWNzSW5mbyI6bnVsbCwiaW5kdklkZW50aXR5IjoiUkVHIiwiaW5jbHVkZVByb21wdGVkU3RhdHVzIjpudWxsfQ==; node_session.sig=JOBGe82eUWsZlO2-zWSfOLK180s',
                                                        'sec-fetch-mode': 'navigate',
                                                        'sec-fetch-site': 'cross-site',
                                                        'upgrade-insecure-requests': '1',
                                                        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.10 Safari/537.36'})
                
                if res != None:
                    for item in res:
                        if item['year'] > end_year or (item['year'] == end_year and item['month'] > end_month):
                            continue
                        views += item['views']
                        citations += item['citations']

            dfx['eid'] = [eid]
            dfx['views'] = [views]
            dfx['citations'] = [citations]

            df_s = df_s.append(dfx, ignore_index=True)

            if index == 200:
                df_s.to_csv(f'SUSTC_Journals/articles_all/all_mendeley_{start_i + 1}-{end_i}_test.csv', index=False)

    df_s.to_csv(f'SUSTC_Journals/articles_all/all_mendeley_{start_i + 1}-{end_i}.csv', index=False)
    return

In [6]:
%%time
step = 10000
t_num = int(len(df) / step)

threads = []
for i in range(0, t_num):
    start_i = i*step
    end_i = start_i + step

    if end_i + step > len(df):
        end_i = len(df)
    
    threadx = threading.Thread(target=grab_mendeley_views, name=f'Thread{start_i}-{end_i}', args=('eid', start_i, end_i, ))
    threadx.start()
    threads.append(threadx)

for t in threads:
    t.join()

00

0
00

0
00
0
0

00

0
0
0
0
00
0

0
0
0
0
0
00
0

0
0
0
00

0
0
0
ReadTimeout: HTTPSConnectionPool(host='www.mendeley.com', port=443): Read timed out. (read timeout=10)
500
500
500
500
500
500
500
ReadTimeout: HTTPSConnectionPool(host='www.mendeley.com', port=443): Read timed out. (read timeout=10)
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
ReadTimeout: HTTPSConnectionPool(host='www.mendeley.com', port=443): Read timed out. (read timeout=10)
500
500
500
1000
500
ReadTimeout: HTTPSConnectionPool(host='www.mendeley.com', port=443): Read timed out. (read timeout=10)
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1500
1000
1000
1000
1000
1000
1000
ReadTimeout: HTTPSConnectionPool(host='www.mendeley.com', port=443): Read timed out. (read timeout=10)
1000
1500
1000
1000
1500
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1500
1500
1500
1500
1500
1500
1500
1000
1000
2000
1500
1500
1500
1500
2000
1500
2000
1500
1500
